# チュートリアル
このノートブックでは，SageMaker上での機械学習モデルの構築・デプロイを以下の流れで説明していきます．

1. ロールの作成
2. データセットの準備
3. 学習
4. デプロイ
5. 推論
6. エンドポイントの削除

In [1]:
import re
import io
import os
import sys
import time
import json
import torch
import pandas as pd
import numpy as np
import boto3
import sagemaker

## 1. ロールの作成

AmazonSageMakerFullAccessを持ったIAMロールを取得します．

- SageMakerのノートブックインスタンス上の場合
    - `role = get_execution_role()` で取得します
- オンプレのjupyter notebook上の場合
    - AmazonSageMakerFullAccessを許可したIAMロールを発行する
    - `role = 'arn:aws:iam::[12桁のAWS ID]:role/[ロール名]'` で取得します

In [2]:
from sagemaker import get_execution_role
role = get_execution_role()

## 2. データセットの準備

データセットを準備する一番簡単な方法は，データをs3にアップロードすることです．
SageMakerのノートブックインスタンス上では，以下のコマンドで簡単にアップロードすることができます．

```
sagemaker.Session()
s3_dataset_path = sess.upload_data(path=[ディレクトリ/ファイルのパス], key_prefix=[s3でのキー])
```

ローカルのnotebookでも，権限を持つs3バケットにアップロードし，`s3_dataset_path = 's3://[バケット名]/[キー]'`とすればOKです．

In [3]:
dataset_dir = './dataset'

In [4]:
# Check datasets.
reviews = pd.read_csv(os.path.join(dataset_dir, '10000_review.csv'))
sentences = pd.read_csv(os.path.join(dataset_dir, '10000_sentence.csv'))
embeddings = np.load(os.path.join(dataset_dir, '10000_embedding.npy'))

In [5]:
reviews.head(3)

,review_id,product_id,product_title,star_rating,review_headline,review_body
0,RDIJS7QYB6XNR,B00EDBY7X8,Monopoly Junior Board Game,5.0,Five Stars,Excellent!!!
1,R36ED1U38IELG8,B00D7JFOPC,56 Pieces of Wooden Train Track Compatible wit...,5.0,Good quality track at excellent price,Great quality wooden track (better than some o...
2,R1UE3RPRGCOLD,B002LHA74O,Super Jumbo Playing Cards by S&S Worldwide,2.0,Two Stars,Cards are not as big as pictured.


In [6]:
sentences.head(3)

,review_id,sentence
0,R36ED1U38IELG8,great quality wooden track better than some ot...
1,R36ED1U38IELG8,perfect match to the various vintages of thoma...
2,R36ED1U38IELG8,there is enough track here to have fun and get...


In [7]:
# Upload datasets to Amazon S3.
sess = sagemaker.Session()
s3_dataset_path = sess.upload_data(path=dataset_dir, key_prefix='data')

print(f"Training data is uploaded to {s3_dataset_path}")

Training data is uploaded to s3://sagemaker-us-west-2-010942746803/data


## 3. 学習
学習を行うためには，以下の2つを最低限準備する必要があります．(これら2つは`source_dir`で指定したディレクトリ直下に保存します．)
- `entry_point.py`
- `requirements.txt`

`source_dir`直下に他のファイル等を置いておけば，学習用のインスタンスではカレントディレクトリ直下にコピーされるため．`entry_point.py`内の処理で操作することが可能です．今回の例では，`modules.pickle`を置いておき，`entry_point.py`で操作しています．ただし，**デプロイ先の推論エンドポイントへはコピーされない**ため，推論時も必要なものを置くことはできません．

また，独自で作成したモジュールは，`dependencies`にリスト形式で指定してください．これにより，学習/推論時にモジュールを利用することができます．この例では，`search`モジュールを作成・利用しています．

`entry_point.py`の詳しい書き方に関しては，解説記事を参照してください．

In [8]:
from sagemaker.pytorch import PyTorch

# Create estimator.
estimator = PyTorch(
    entry_point='entry_point.py',
    source_dir='source_dir',
    dependencies=['search'],
    role=role,
    py_version='py3',
    framework_version='1.9.0',
    train_instance_count=1,
    train_instance_type='ml.m4.xlarge'
)

In [9]:
# Train.
estimator.fit({'train': s3_dataset_path})

2020-01-25 10:55:24 Starting - Starting the training job...
2020-01-25 10:55:26 Starting - Launching requested ML instances......
2020-01-25 10:56:28 Starting - Preparing the instances for training...
2020-01-25 10:57:18 Downloading - Downloading input data...
2020-01-25 10:57:30 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-25 10:58:29,822 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-25 10:58:29,825 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-25 10:58:29,839 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-01-25 10:58:29,840 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-01-25 10:58:38,597 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2

## 4. デプロイ

SageMaker APIを用いれば，学習済みモデルを簡単にデプロイすることができます．(少し時間がかかります．)

`deploy`関数は`sagemaker.RealTimePredictor`オブジェクトを返し，これを利用して推論することができます．このとき，リクエストのシリアライザとレスポンスのデシリアライザを指定します．指定しない場合，クライアント側でそれらの処理をする必要があります．

In [10]:
# Deploy the trained model.
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge'
)

-------------------!

In [39]:
from sagemaker.predictor import JSONSerializer, JSONDeserializer

predictor.content_types = 'application/json'
predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [40]:
request = {
    'query': 'it still looks brand new too',
    'n_items': 1
}
response = predictor.predict(request)

In [41]:
def print_result(response):
    for value in response.values():
        print('-' * 10)
        for k, v in value.items():
            print(f'- {k}')
            print(f'{v}')

print_result(response)

----------
- review_id
R1V5I8W64XQA8R
- product_id
B00388C3C4
- product_title
Fisher-Price Laugh & Learn Learning Kitchen Activity Center
- star_rating
5.0
- review_headline
One of our most beloved toys, even 1.5 yrs later
- review_body
My son received this as a gift when he was 9 months old. He played with it daily (sometimes 30 min at a time) until he was almost 2, when he graduated to a big kitchen. This was put away for a few months, but recently came out again now that his little sister is 6 months old. It's one of the few things that they can really play with together, one on each side.  My son (now 2.5) loves playing with it with his sis (and she of course loves it too). We have a lot of toys, but this one stands apart as versatile, fun, and extremely long lived! It still looks brand new too.
- product_search_score
0.9975305795669556
- sentence
it still looks brand new too.


## 5. 推論
SageMakerのAPIを利用することで，デプロイしたエンドポイントでの推論を行うことができます．

ここでは，データのシリアライズとデシリアライズはクライアント側で行う必要があります．ここではJSON形式でデータの送受信を行なっているので，送信時に`json.dumps`，受信時に`json.load`を行なっています．

In [42]:
request = {
    'query': 'My children liked it',
    'n_items': 1
}

client = boto3.client('sagemaker-runtime')

response = client.invoke_endpoint(
    EndpointName='pytorch-training-2020-01-25-10-53-51-302', 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(request)
    )

In [43]:
body = json.load(response['Body'])
print_result(body)

----------
- review_id
R2XKMLHEG7Z402
- product_id
B00IGNWYGQ
- product_title
Play-Doh Mix 'n Match Magical Designs Palace Set Featuring Disney Princess Aurora
- star_rating
4.0
- review_headline
Inventive and fun, some parts hard to do
- review_body
My kids loved this. Lots of sparkly play doh and tons of molds. One star comes off because it's hard to get play do to press out of the skirt, and once you're done with that it's tough to get the skirt to come off the little pedestal.
- product_search_score
0.9600517749786377
- sentence
my kids loved this.


## 6. エンドポイントの削除
エンドポイントは，起動してる時間ずっと課金されてしまうので，こまめに削除します．

In [44]:
predictor.delete_endpoint()